In [ ]:
from networks_bf import get_all_devices_combined
from services_bf import get_all_services
import itertools
import numpy as np
import pandas as pd

# Create an empty DataFrame to store the results
columns = ['config', 'physical_machines', 'services', 'generations', 'alpha', 'beta', 'latency', 'energy', 'placement_physical', 'placement_vm', 'is_perfect']
results_headers = pd.DataFrame(columns=columns)
results_headers.to_csv('results_mini_bf.csv', mode='a', index=False)
results_df = pd.DataFrame()

for c in range(1, 101):
    
    print(f"Processing config-{c}....")
    
    physical_machines = get_all_devices_combined(f"config-{c}")
    services = get_all_services(f"config-{c}")
    
    # Define the problem-specific parameters
    n = len(physical_machines)  # Number of physical machines
    s = len(services)  # Number of services
    
    vm_list = []
    
    for p in range(len(physical_machines)):
        for v in range(len(physical_machines[p].guest_machines)):
            vm_list.append([p, v])
    
    # Calculate all permutations of vm_list with s places (with repetition)
    permutations = itertools.product(vm_list, repeat=s)
    
    # Variable number of virtual machines for each physical machine
    max_v = max(map(lambda machine: len(machine.guest_machines), physical_machines))
    
    ################ Physical and virtual machine configurations ##################
    
    # Initialization
    R = np.empty(shape=(n, max_v))  # Request network delay of the machine
    R.fill(999999) # Initialize with very large value
    S = np.empty(shape=(n, max_v))  # Response network delay of the machine
    S.fill(999999) # Initialize with very large value
    X = np.empty(shape=(n, max_v))  # Maximum IPS possible
    X.fill(1) # Initialize with 1
    
    PI = np.empty(shape=(n, max_v))  # Power consumption in idle
    PI.fill(999) # Initialize with very large value
    PM = np.empty(shape=(n, max_v))  # Maximum power consumption
    PM.fill(999) # Initialize with very large value
    
    # Fill in with the actual numbers
    for i in range(len(physical_machines)):
        for j in range(len(physical_machines[i].guest_machines)):
            R[i][j] = physical_machines[i].guest_machines[j].net_delay_request
            S[i][j] = physical_machines[i].guest_machines[j].net_delay_response
            X[i][j] = physical_machines[i].guest_machines[j].max_instructions_per_second
            PI[i][j] = physical_machines[i].guest_machines[j].idle_cpu_utilization
            PM[i][j] = physical_machines[i].guest_machines[j].max_cpu_utilization
    
    ################ Service configurations ###################
    
    # Acceptable latencies for services
    SAL = [service.acceptable_latency for service in services]
    # Initialize IPS values for services (replace with actual data)
    SIPS = [service.average_instructions_per_second for service in services]
    # Layers considered for service
    SL = [service.layer for service in services]
    
    perm_list = list(permutations)
    
    # Define minimum state variables
    min_latency = float("inf")
    min_energy = float("inf")
    min_objective = float("inf")
    physical_placement = []
    vm_placement = []
    is_perfect = []
    
    for perm in perm_list:
        
        perfect = [True] * s # To check if the permutation is perfect
        
        I = np.empty(shape=(n, max_v))  # IPS currently executed
        I.fill(0) # Initialize with 0
        # Calculate energy consumption based on the parameters
        E = PI + (PM - PI) * (I / X)
    
        total_latency = 0
        total_energy = 0
        
        least_latency = float("inf")
        least_latency_index = []
        for p in range(len(physical_machines)):
                    for v in range(len(physical_machines[p].guest_machines)):
                        total_energy += E[p][v]
                        if least_latency > R[p][v] + S[p][v]:
                            least_latency = R[p][v] + S[p][v]
                            least_latency_index = [p, v]
        
        invalid = False # Discard the permutation if it is invalid
        
        for i in range(s):
            p = perm[i][0]
            v = perm[i][1]
            
            vm = physical_machines[p].guest_machines[v]
            
            if SL[i] is not None and SL[i] != vm.layer: # Consider the layer constrains provided by the service
                invalid = True
                break
            
            latency = R[p][v] + (SIPS[i] * 1000 / X[p][v]) + S[p][v]
            
            pm_idx = p
            vm_idx = v
            
            # If this configuration is not perfect, then place it on the least latency vm
            if latency > SAL[i]:
                perfect[i] = False
                pm_idx = least_latency_index[0]
                vm_idx = least_latency_index[1]
                
            total_latency += latency
            I[pm_idx][vm_idx] = I[pm_idx][vm_idx] + SIPS[i] # Update the current instructions per second
            energy = PI[pm_idx][vm_idx] + (PM[pm_idx][vm_idx] - PI[pm_idx][vm_idx]) * (I[pm_idx][vm_idx] / X[pm_idx][vm_idx])
            total_energy += energy  # Calculate energy consumption
            
        
        # Something abnormally wrong happened
        if invalid:
            continue
        
        objective = 0.5 * total_latency + 0.5 * total_energy # Only alpha = 0.5 is considered
        if min_objective > objective:
            min_objective = objective
            physical_placement = [elem[0] for elem in perm]
            vm_placement = [elem[1] for elem in perm]
            min_latency = total_latency
            min_energy = total_energy
            is_perfect = perfect
            
    # Append the data to the results DataFrame
    results_df = results_df._append({'config': f'config-{c}',
                                     'physical_machines': n,
                                     'services': s,
                                     'generations': 0,
                                     'alpha': 0.5,
                                     'beta': 0.5,
                                     'latency': min_latency,
                                     'energy': min_energy,
                                     'placement_physical': physical_placement,
                                     'placement_vm': vm_placement,
                                     'is_perfect': is_perfect}, ignore_index=True)

    results_df.to_csv('results_mini_bf.csv', mode='a', index=False, header=False)
    results_df = pd.DataFrame()

Processing config-1....
Processing config-2....
Processing config-3....
Processing config-4....
